# Deploying a Streamlit Image Classification App on Google Cloud Run

This guide provides you with the scaffold and steps to build a simple image classification application using Streamlit.


### What is Streamlit?
[Streamlit](https://streamlit.io/) is an open-source Python library that's become the go-to tool for quickly creating interactive web applications, especially for data science and machine learning projects.  Streamlit's intuitive design makes it easy to build powerful apps with minimal code.

Check out the amazing examples in the [Streamlit Gallery](https://streamlit.io/gallery) to see the wide range of applications you can create. From data dashboards and visualizations to machine learning demos and interactive tools!

This notebook doesn't require an accelerator, but optionally you can attach a GPU to speed up the training process below.<br>
(It takes more than 15 minutes to train without a GPU. Or you can stop the training in the middle just to simply deploy the application.)

## Set up

In [ ]:
import os
import warnings

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore")

In [ ]:
import base64
import json
import shutil
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import tensorflow as tf
import tensorflow_hub as hub
from google.cloud import aiplatform
from oauth2client.client import GoogleCredentials

## Building an Image Classification Model

We'll create a flower image classification model using the 5-Flower dataset as an example.

If you want to know more about the code below, please check [../../../notebooks/image_models/solutions/3_tf_hub_transfer_learning.ipynb](https://github.com/GoogleCloudPlatform/asl-ml-immersion/blob/master/notebooks/image_models/solutions/3_tf_hub_transfer_learning.ipynb).

In [ ]:
PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]
BUCKET = PROJECT + "-flowers"
FILE_DIR = f"gs://{BUCKET}/data"
REGION = "us-central1"

os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION

CLASSES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

BATCH_SIZE = 32

In [ ]:
%%bash
exists=$(gsutil ls -d | grep -w gs://${BUCKET}/)

if [ -n "$exists" ]; then
    echo -e "Bucket exists, let's not recreate it."
    
else
    echo "Creating a new GCS bucket."
    gsutil mb -l ${REGION} gs://${BUCKET}
    echo "Here are your current buckets:"
    gsutil ls
fi

In [ ]:
!gsutil cp gs://asl-public/data/flowers/tfrecords/* {FILE_DIR}

In [ ]:
TRAIN_PATTERN = FILE_DIR + "/train*"
EVAL_PATTERN = FILE_DIR + "/eval*"


def parse_example(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.io.decode_jpeg(example["image"], channels=3)
    example["image"] = tf.image.resize(
        example["image"], [IMG_HEIGHT, IMG_WIDTH]
    )
    example["image"] = example["image"] / 255
    return example["image"], example["label"]


train_ds = (
    tf.data.TFRecordDataset(tf.io.gfile.glob(TRAIN_PATTERN))
    .map(parse_example)
    .batch(BATCH_SIZE)
)
eval_ds = (
    tf.data.TFRecordDataset(tf.io.gfile.glob(EVAL_PATTERN))
    .map(parse_example)
    .batch(10)
)

In [ ]:
module_selection = "mobilenet_v2_100_224"
module_handle = "https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(
    module_selection
)

transfer_model = tf.keras.Sequential(
    [
        hub.KerasLayer(module_handle, trainable=True),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(
            len(CLASSES),
            activation="softmax",
            kernel_regularizer=tf.keras.regularizers.l2(0.0001),
        ),
    ]
)

transfer_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
transfer_model.fit(
    train_ds,
    epochs=5,
    validation_data=eval_ds,
)

In [ ]:
shutil.rmtree("export", ignore_errors=True)
os.mkdir("export")
transfer_model.save("export/flowers_model")

In [ ]:
!ls export/flowers_model

---

## Build a streamlit application

With our image classification model prepared, let's create a Streamlit app to make it interactive and user-friendly.

### Import Libraries
First, we'll import the necessary Python libraries, including Streamlit itself, as well as any other libraries needed for image processing and model loading.

In [ ]:
%%writefile app.py
"""Streamlit Image Classification App"""

import numpy as np
import streamlit as st
import tensorflow as tf
from tensorflow.keras.models import load_model

### Add text elements
Streamlit eventually renders a web page, but we can simply use Python modules to define and configure the the web page elements.

We'll start by configuring some metadata for our app.

- [`st.set_page_config`](https://docs.streamlit.io/develop/api-reference/configuration/st.set_page_config) lets us customize aspects like the page title and favicon (the little icon that appears in your browser tab).

Streamlit provides a variety of ways to display text content:
- [`st.title`](https://docs.streamlit.io/develop/api-reference/text/st.title) is used to add a main heading to our page.
- You can also use [`st.header`](https://docs.streamlit.io/develop/api-reference/text/st.header) and [`st.subheader`](https://docs.streamlit.io/develop/api-reference/text/st.subheader) for smaller headings.
- [`st.text`](https://docs.streamlit.io/develop/api-reference/text/st.text) is for displaying plain text.
- [`st.markdown`](https://docs.streamlit.io/develop/api-reference/text/st.markdown) lets you add formatted text using Markdown syntax.
For more options, you can check the Streamlit documentation for [other text elements](https://docs.streamlit.io/develop/api-reference/text).

Streamlit also offer a "swiss-army knife" command called [`st.write`](https://docs.streamlit.io/develop/api-reference/write-magic/st.write). It can handle many types of content, including text, DataFrames (tables of data), Matplotlib plots, and even Keras machine learning models.

Also, here we define a few global variables that we'll use later.

In [ ]:
%%writefile -a app.py

st.set_page_config(page_title="5-Flower Classifier", page_icon="🌷")

st.title("5-Flower Classifier")

st.markdown(
    "Welcome to this simple web application that classifies 5 flowers"
    + "(daisy, dandelion, roses, sunflowers, tulips)."
)

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3
CLASSES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

### Defining Model Loading with Caching

We'll define a function to load our image classification model using `keras.models.load_model`.

To avoid reloading the model every time we make a prediction, we'll use the [`@st.cache_resource`](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.cache_resource) decorator. This decorator caches the output of our function, making it much faster to access the model on subsequent runs.

**Note: [`@st.cache_resource`](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.cache_resource) is best suited for global objects that can't be easily serialized (converted to a simple data format), such as database connections or complex machine learning models. For simpler, serializable objects (like pandas DataFrames), you might consider using [`tf.cache_data`](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.cache_data) instead.**

For a deeper dive into caching and how to optimize your Streamlit apps, you can check out [this document](https://docs.streamlit.io/develop/concepts/architecture/caching).

In [ ]:
%%writefile -a app.py

@st.cache_resource(show_spinner=False)
def load_and_cache_model():
    model_path = "flowers_model"
    model = load_model(model_path)
    return model


### Defining Utility Functions

We also define a few utility functions .

To prepare images for our model and interpret its predictions, we'll define a few helper functions for model serving, like image decoding and rescaling, resizing, and post-processing.

In [ ]:
%%writefile -a app.py

def read_image(img_bytes):
    img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img


def predict(model, image):
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = np.expand_dims(image, axis=0)
    predictions = model.predict(image)
    pred_index = np.argmax(predictions[0])
    return predictions[0][pred_index], CLASSES[pred_index]

### Defining the Application Logic

Now let's  the main application logic.

We design an application that has these capabilities:
1. **Upload image**: We'll use [`st.file_uploader`](https://docs.streamlit.io/develop/api-reference/widgets/st.file_uploader) to create a widget that allows users to select and upload an image file. We can specify allowed file types (like PNG or JPG) to guide the user
2. **Show the uploaded image**: Once an image is uploaded, we'll display it using [`st.image`](https://docs.streamlit.io/develop/api-reference/media/st.image). Alternatively, the versatile [`tf.write`](https://docs.streamlit.io/develop/api-reference/write-magic/st.write) function can also handle image display.
3. **Start image classification**: We'll create a button labeled "Classify" using [`st.button`](https://docs.streamlit.io/develop/api-reference/widgets/st.button), which returns `True` when it is pushed. (For more advanced button use cases, like stateful buttons, you can refer to the Streamlit documentation: https://docs.streamlit.io/develop/concepts/design/buttons).
4. **Call the classification model**: We'll use our previously defined functions to load the model and make a prediction. Since this might take a moment, we'll wrap it in a with [`with st.spinner`](https://docs.streamlit.io/develop/api-reference/status/st.spinner) block to provide visual feedback to the user that the process is underway.
5. **Show the result**: Once the model generates a prediction, we'll display it to the user. We'll use [`st.success`](https://docs.streamlit.io/develop/api-reference/status/st.success) to clearly indicate that the classification was completed successfully, along with the predicted flower type.



In [ ]:
%%writefile -a app.py

def main():
    file_uploaded = st.file_uploader("Choose File", type=["png", "jpg", "jpeg"])
    if file_uploaded is not None:
        image = read_image(file_uploaded.read())
        st.image(image.numpy(), caption="Uploaded Image", use_column_width=True)
        class_btn = st.button("Classify")
        if class_btn:
            with st.spinner("Model predicting...."):
                loaded_model = load_and_cache_model()
                prob, prediction = predict(loaded_model, image)
                st.success(f"Prediction: {prediction} - {prob:.2%}")


if __name__ == "__main__":
    main()

## Deploying the App on Cloud Run
Our Streamlit application is ready! Let's deploy it to Google Cloud Run, a serverless platform designed to run containerized applications seamlessly.

### Defining Dockerfile and Dependencies
To containerize our app for Cloud Run, we define `requirements.txt` and `Dockerfile`.

In [ ]:
%%writefile requirements.txt
numpy==1.23.5
streamlit==1.30.0
tensorflow==2.12.0

In [ ]:
%%writefile Dockerfile
FROM python:3.10.14

WORKDIR /app

COPY requirements.txt /app
RUN pip install -r requirements.txt

COPY export /app
COPY app.py /app

EXPOSE 8080

CMD streamlit run --server.port 8080 --server.enableCORS false app.py

**Note: We've split the `COPY` command into multiple lines, each copying different files. Although this is not required, this is a crucial optimization for Docker's caching mechanism.<br> If you make changes only to app.py, the next time you build the image, Docker will reuse the cached layers for the dependency installation and other files, speeding up the build process significantly.**

### Building and Pushing the Container to Artifact Registry
Now that we have our `Dockerfile`, we can build the Docker image of our Streamlit app and push it to Google Cloud's Artifact Registry. 
Artifact Registry offers a secure and scalable way to store your container images.

First, we'll create a new repository in Artifact Registry to house our container image.

In [ ]:
STREAMLIT_ARTIFACT_REG_REPO = "flower-classification-app"
os.environ["STREAMLIT_ARTIFACT_REG_REPO"] = STREAMLIT_ARTIFACT_REG_REPO

In [ ]:
%%bash
if ! gcloud artifacts repositories describe $STREAMLIT_ARTIFACT_REG_REPO \
       --location=$REGION > /dev/null 2>&1; then
    gcloud artifacts repositories create $STREAMLIT_ARTIFACT_REG_REPO \
        --project=$PROJECT --location=$REGION --repository-format=docker
fi

### Defining cloudbuild.yaml for Cloud Build
We'll use Google Cloud Build to automate the process of building our Docker image and pushing it to Artifact Registry. 

Cloud Build is a serverless CI/CD platform that lets you define build steps in a configuration file called cloudbuild.yaml.

In [ ]:
CONTAINER_PATH = (
    f"us-central1-docker.pkg.dev/{PROJECT}/{STREAMLIT_ARTIFACT_REG_REPO}/app"
)
os.environ["CONTAINER_PATH"] = CONTAINER_PATH

Here's a cloudbuild.yaml file that incorporates caching to make our builds faster:

1. **Pull Existing Image**: The first step attempts to pull the latest version of your Docker image from Artifact Registry. Here we use `bash -c` command as the entrypoint so that the job can ignore and proceed even if this step fails in the first run.
2. **Build with Caching**: The second step builds the new image. `--cache-from` flag tells Docker to use the layers from the pulled image as a cache, speeding up the build if there are no changes to those layers.

In [ ]:
%%writefile cloudbuild.yaml
steps:
- name: 'gcr.io/cloud-builders/docker'
  entrypoint: 'bash'
  args: ['-c', 'docker pull ${_CONTAINER_PATH}:latest || exit 0']
- name: 'gcr.io/cloud-builders/docker'
  args: [
            'build',
            '-t', '${_CONTAINER_PATH}:latest',
            '--cache-from', '${_CONTAINER_PATH}:latest',
            '.'
        ]
images: ['${_CONTAINER_PATH}:latest']

### Building the Container Image

With our cloudbuild.yaml file defined, we can now instruct Cloud Build to construct our Docker image.

In [ ]:
!gcloud builds submit --config cloudbuild.yaml --region $REGION . --substitutions _CONTAINER_PATH={CONTAINER_PATH}

### Deploying to Cloud Run
With our container image stored in Artifact Registry is ready, we're all set to deploy our Streamlit app to Cloud Run.

You can also consider incorporating this command into the `cloudbuild.yaml` we defined above.

In [ ]:
APP_NAME = "flower-classification"
os.environ["APP_NAME"] = APP_NAME

In [ ]:
%%bash
echo 'Deploying the application to Cloud Run...'
gcloud run deploy $APP_NAME \
  --image $CONTAINER_PATH:latest --min-instances 1 --max-instances 1 --cpu 1 \
  --memory 4Gi --region us-central1 > /dev/null 2>&1 && \
echo 'Deployment Done.'

### Connect to Cloud Run app via Cloud Shell


You have a lot of flexibility when it comes to configuring access to your Cloud Run service. You can even [make it publicly accessible](https://cloud.google.com/run/docs/authenticating/public) if you want to.

However, for this example, let's see how to connect to your Cloud Run app securely from Cloud Shell using a proxy.

Follow these steps to open the app from Cloud Shell.
1. Run the next cell, copy the output `gcloud run services proxy ...`command.
2. Open Cloud Shell, paste and run the command.
3. In Cloud Shell, click the "Web Preview" button on the toolbar.
4. Select "Preview on port 8080"
5. A new browser tab or window will open, displaying your Streamlit app.

In [ ]:
print(
    f"gcloud run services proxy {APP_NAME} --project {PROJECT} --region {REGION}"
)

## Prediction
Now you're ready to test your model.

Search for a clear image of one of the flower types your model recognizes (daisy, dandelion, rose, sunflower, or tulip), upload your image and click "Classify." <br>
The app will display the predicted flower type!

The first access and prediction may take some time, but it'll be faster from the second time thanks to the cache.

<img width="1004" alt="image" src="https://github.com/GoogleCloudPlatform/asl-ml-immersion/assets/6895245/4b6fd594-69fd-491e-9ca2-c199d3cb66a7">


Copyright 2024 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.